In [5]:
from keras import backend as K

ModuleNotFoundError: No module named 'keras'

In [4]:
import numpy as np
import pandas as pd

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import keras

ModuleNotFoundError: No module named 'keras'

# Random Example

In [9]:
model = Sequential([
    Dense(100, input_shape=(2,), activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(2, activation='softmax'),
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

train_data = np.array([
    [1, 1],
    [1, 0],
    [0, 1],
    [0, 0]
])

# X AND Y
train_labels = keras.utils.to_categorical(np.array([
    1,
    0,
    0,
    1
]), num_classes=2)

model.summary()


NameError: name 'Sequential' is not defined

In [10]:
model.fit(train_data, train_labels, epochs=10, verbose=False)
model.evaluate(train_data, train_labels)

NameError: name 'model' is not defined

In [11]:
model.predict(train_data)

NameError: name 'model' is not defined

# Softmax Regression

## Try to Achieve This
`X and (Y OR Z)`

In [12]:
model = Sequential([
    Dense(500, input_shape=(3,), activation='sigmoid'),
    Dense(2, activation='sigmoid'),
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

NameError: name 'Sequential' is not defined

In [13]:
train_data = np.array([
    [1, 1, 1],
    [1, 1, 0],
    [1, 0, 1],
    [1, 0, 0],
    [0, 1, 1],
    [0, 1, 0],
    [0, 0, 1],
    [0, 0, 0],
])

# X AND Y
train_labels = keras.utils.to_categorical(np.array([
    1,
    1,
    1,
    0,
    0,
    0,
    0,
    0
]), num_classes=2)

NameError: name 'keras' is not defined

In [14]:
model.fit(train_data, train_labels, epochs=100, verbose=True)

NameError: name 'model' is not defined

# Indodax Data Retrieval

In [34]:
import requests
import datetime
import pytz
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt

In [55]:
root_uri_trade_api = 'https://indodax.com/api'
root_uri_chart_api = 'https://indodax.com/tradingview/history'
#?symbol=BTCIDR&resolution=15&from=1546489788&to=1547785848'
chart_currency = 'BTCIDR'
period = 15
time_begin = int(datetime.datetime(2018, 1, 1).timestamp())
time_end = int(datetime.datetime(2018, 12, 31).timestamp())
chart_query = '?symbol={}&resolution={}&from={}&to={}'.format(chart_currency, period, time_begin, time_end)

r = requests.get(root_uri_chart_api + chart_query)
price = pd.DataFrame({
    'open': r.json()['o'],
    'close': r.json()['c'],
    'high': r.json()['h'],
    'low': r.json()['l'],
    'volume': r.json()['v']
# }, index=list(map(lambda t: (datetime.datetime.utcfromtimestamp(t) + datetime.timedelta(hours=7)).strftime('%d/%m %H.%M'), r.json()['t'])))
}, index=list(map(lambda t: (datetime.datetime.utcfromtimestamp(t) + datetime.timedelta(hours=7)), r.json()['t'])))

In [46]:
candle_color = lambda o, c: 'g' if c >= o else 'r'
fig_w = len(price.open)//5
fig_h = fig_w // 2
plt.figure(figsize=(fig_w, fig_h))
plt.bar( [i for i in range(len(price.open))],
        [abs(c-o) for c,o in zip(price.close, price.open)],
        [1 for i in range(len(price.open))],
        [min(c, o) for c,o in zip(price.close, price.open)],
        color=[candle_color(o, c) for c,o in zip(price.close, price.open)]
)
plt.bar( [i for i in range(len(price.high))],
        [h-l for h,l in zip(price.high, price.low)],
        [.2 for i in range(len(price.high))],
        [l for l in price.low],
        color=[candle_color(o, c) for c,o in zip(price.close, price.open)]
)
plt.plot([i for i in range(len(price.high))], price.close.ewm(span=10).mean(), color='blue')
plt.plot([i for i in range(len(price.high))], price.close.ewm(span=30).mean(), color='orange')
ax = plt.gca()
ax.set_xticks([i for i in range(len(price.index))][::10])
ax.set_xticklabels(price.index[::10], rotation=-45)
plt.figure(figsize=(fig_w, fig_h))
plt.bar([i for i in range(len(price.volume))],
        price.volume,
        .5,
        color='blue'
)
ax = plt.gca()
ax.set_xticks([i for i in range(len(price.index))][::10])
ax.set_xticklabels(price.index[::10], rotation=-45)

use this to visualize
plt.show()

KeyboardInterrupt: 

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f24f9b6d950> (for post_execute):


ValueError: Image size of 377352x188640 pixels is too large. It must be less than 2^16 in each direction.

ValueError: Image size of 377352x188640 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 377352x188640 with 1 Axes>

In [56]:
price

,open,close,high,low,volume
2018-01-01 00:00:00,224501000,222697000,224503000,222673000,3.869549
2018-01-01 00:15:00,222697000,222899000,223083000,222697000,3.047817
2018-01-01 00:30:00,222899000,222867000,223083000,222000000,3.944302
2018-01-01 00:45:00,222867000,222576000,223000000,222352000,1.937695
2018-01-01 01:00:00,222576000,224199000,224200000,222575000,1.799813
2018-01-01 01:15:00,224199000,223163000,224201000,222500000,1.677477
2018-01-01 01:30:00,223163000,223025000,223490000,223025000,1.265841
2018-01-01 01:45:00,223025000,223026000,223026000,223025000,0.334693
2018-01-01 02:00:00,223026000,223999000,223999000,223025000,0.930138
2018-01-01 02:15:00,223999000,224722000,224810000,223751000,1.532012


In [57]:
action_list = {}
action_list['\\'] = {'action'}
action_list['action'] = []

# weird workaround due to limitation to column name, see below
# HOLD --> 0
# BUY --> -1
# SELL --> 1
for i in range(len(price.index)-1):
    current = price.iloc[i]
    before = price.iloc[i-1]
    after = price.iloc[i+1]
    if current.close < after.close:
        if current.close > before.close:
            action_list['action'].append(0)
        else:
            action_list['action'].append(-1)
    elif current.close > after.close:
        if current.close < before.close:
            action_list['action'].append(0)
        else:
            action_list['action'].append(1)
    else:
        action_list['action'].append(0)
action_list['action'].append(0)
action_frame = pd.DataFrame(action_list['action']).set_index(price.index)

# we need the stock price somehow
action_frame = action_frame.join(price)

# column names change 'Action' to 'low'
# weird workaround due to limitation to column name
# reference : https://www.backtrader.com/docu/pandas-datafeed/pandas-datafeed.html?highlight=pandasdata
action_frame.columns = ['low', 'open', 'close', 'high', 'real_low', 'volume']

In [58]:
action_frame

,low,open,close,high,real_low,volume
2018-01-01 00:00:00,0,224501000,222697000,224503000,222673000,3.869549
2018-01-01 00:15:00,1,222697000,222899000,223083000,222697000,3.047817
2018-01-01 00:30:00,0,222899000,222867000,223083000,222000000,3.944302
2018-01-01 00:45:00,-1,222867000,222576000,223000000,222352000,1.937695
2018-01-01 01:00:00,1,222576000,224199000,224200000,222575000,1.799813
2018-01-01 01:15:00,0,224199000,223163000,224201000,222500000,1.677477
2018-01-01 01:30:00,-1,223163000,223025000,223490000,223025000,1.265841
2018-01-01 01:45:00,0,223025000,223026000,223026000,223025000,0.334693
2018-01-01 02:00:00,0,223026000,223999000,223999000,223025000,0.930138
2018-01-01 02:15:00,1,223999000,224722000,224810000,223751000,1.532012


In [59]:
import backtrader as bt
import backtrader.feeds as btfeeds

In [60]:
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # weird workaround due to limitation to column name
        # we take the low column as action
        self.data_action = self.datas[0].low
        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # buy!
            if self.data_action[0] == -1:
                self.log('BUY CREATE')
                self.order = self.buy()

        else:
            # sell!
            if self.data_action[0] == 1:
                self.log('SELL CREATE')
                self.order = self.sell()

In [61]:
# data_feed = btfeeds.PandasData(dataname=price, openinterest=None)
data_feed = btfeeds.PandasData(dataname=action_frame, openinterest=None)
#print(data_feed)

In [62]:
cerebro = bt.Cerebro()
cerebro.addstrategy(TestStrategy)
cerebro.adddata(data_feed)
# cerebro.adddata(action_frame)
cerebro.broker.setcash(10000000000.0)

# Print out the starting conditions
startingPortofolio = cerebro.broker.getvalue()
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
finalPortofolio = cerebro.broker.getvalue()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Profit: %.2f (%.6f%%)' % (finalPortofolio - startingPortofolio, (finalPortofolio - startingPortofolio)/startingPortofolio*100))

Starting Portfolio Value: 10000000000.00
2018-01-01, BUY CREATE
2018-01-01, BUY EXECUTED, 222576000.00
2018-01-01, SELL CREATE
2018-01-01, SELL EXECUTED, 224199000.00
2018-01-01, BUY CREATE
2018-01-01, BUY EXECUTED, 223025000.00
2018-01-01, SELL CREATE
2018-01-01, SELL EXECUTED, 224722000.00
2018-01-01, BUY CREATE
2018-01-01, BUY EXECUTED, 222102000.00
2018-01-01, SELL CREATE
2018-01-01, SELL EXECUTED, 224288000.00
2018-01-01, BUY CREATE
2018-01-01, BUY EXECUTED, 222809000.00
2018-01-01, SELL CREATE
2018-01-01, SELL EXECUTED, 223757000.00
2018-01-01, BUY CREATE
2018-01-01, BUY EXECUTED, 223701000.00
2018-01-01, SELL CREATE
2018-01-01, SELL EXECUTED, 224250000.00
2018-01-01, BUY CREATE
2018-01-01, BUY EXECUTED, 223324000.00
2018-01-01, SELL CREATE
2018-01-01, SELL EXECUTED, 223327000.00
2018-01-01, BUY CREATE
2018-01-01, BUY EXECUTED, 222532000.00
2018-01-01, SELL CREATE
2018-01-01, SELL EXECUTED, 223645000.00
2018-01-01, BUY CREATE
2018-01-01, BUY EXECUTED, 222593000.00
2018-01-01, SEL

2018-01-07, SELL EXECUTED, 247497000.00
2018-01-07, BUY CREATE
2018-01-07, BUY EXECUTED, 245689000.00
2018-01-07, SELL CREATE
2018-01-07, SELL EXECUTED, 246900000.00
2018-01-07, BUY CREATE
2018-01-07, BUY EXECUTED, 245899000.00
2018-01-07, SELL CREATE
2018-01-07, SELL EXECUTED, 248500000.00
2018-01-07, BUY CREATE
2018-01-07, BUY EXECUTED, 247714000.00
2018-01-07, SELL CREATE
2018-01-07, SELL EXECUTED, 248350000.00
2018-01-07, BUY CREATE
2018-01-07, BUY EXECUTED, 247698000.00
2018-01-07, SELL CREATE
2018-01-07, SELL EXECUTED, 248990000.00
2018-01-07, BUY CREATE
2018-01-07, BUY EXECUTED, 248101000.00
2018-01-07, SELL CREATE
2018-01-07, SELL EXECUTED, 248800000.00
2018-01-07, BUY CREATE
2018-01-07, BUY EXECUTED, 245699000.00
2018-01-07, SELL CREATE
2018-01-07, SELL EXECUTED, 248788000.00
2018-01-07, BUY CREATE
2018-01-07, BUY EXECUTED, 248701000.00
2018-01-07, SELL CREATE
2018-01-07, SELL EXECUTED, 248799000.00
2018-01-07, BUY CREATE
2018-01-07, BUY EXECUTED, 247499000.00
2018-01-07, SELL

2018-01-13, BUY EXECUTED, 216503000.00
2018-01-13, SELL CREATE
2018-01-13, SELL EXECUTED, 217007000.00
2018-01-13, BUY CREATE
2018-01-13, BUY EXECUTED, 216505000.00
2018-01-13, SELL CREATE
2018-01-13, SELL EXECUTED, 216700000.00
2018-01-13, BUY CREATE
2018-01-13, BUY EXECUTED, 216270000.00
2018-01-13, SELL CREATE
2018-01-13, SELL EXECUTED, 216597000.00
2018-01-13, BUY CREATE
2018-01-13, BUY EXECUTED, 216492000.00
2018-01-13, SELL CREATE
2018-01-13, SELL EXECUTED, 217901000.00
2018-01-13, BUY CREATE
2018-01-13, BUY EXECUTED, 217522000.00
2018-01-13, SELL CREATE
2018-01-13, SELL EXECUTED, 217676000.00
2018-01-13, BUY CREATE
2018-01-13, BUY EXECUTED, 216028000.00
2018-01-13, SELL CREATE
2018-01-13, SELL EXECUTED, 216502000.00
2018-01-13, BUY CREATE
2018-01-13, BUY EXECUTED, 216176000.00
2018-01-13, SELL CREATE
2018-01-13, SELL EXECUTED, 217480000.00
2018-01-13, BUY CREATE
2018-01-13, BUY EXECUTED, 216005000.00
2018-01-13, SELL CREATE
2018-01-13, SELL EXECUTED, 216898000.00
2018-01-13, BUY

2018-01-19, SELL EXECUTED, 162709000.00
2018-01-19, BUY CREATE
2018-01-19, BUY EXECUTED, 155700000.00
2018-01-19, SELL CREATE
2018-01-19, SELL EXECUTED, 156591000.00
2018-01-19, BUY CREATE
2018-01-19, BUY EXECUTED, 153369000.00
2018-01-19, SELL CREATE
2018-01-19, SELL EXECUTED, 157119000.00
2018-01-19, BUY CREATE
2018-01-19, BUY EXECUTED, 156151000.00
2018-01-19, SELL CREATE
2018-01-19, SELL EXECUTED, 158305000.00
2018-01-19, BUY CREATE
2018-01-19, BUY EXECUTED, 157502000.00
2018-01-19, SELL CREATE
2018-01-19, SELL EXECUTED, 161000000.00
2018-01-19, BUY CREATE
2018-01-19, BUY EXECUTED, 159899000.00
2018-01-19, SELL CREATE
2018-01-19, SELL EXECUTED, 163000000.00
2018-01-19, BUY CREATE
2018-01-19, BUY EXECUTED, 159499000.00
2018-01-19, SELL CREATE
2018-01-19, SELL EXECUTED, 161744000.00
2018-01-19, BUY CREATE
2018-01-19, BUY EXECUTED, 161606000.00
2018-01-19, SELL CREATE
2018-01-19, SELL EXECUTED, 163617000.00
2018-01-19, BUY CREATE
2018-01-19, BUY EXECUTED, 161882000.00
2018-01-19, SELL

2018-01-25, SELL EXECUTED, 153558000.00
2018-01-25, BUY CREATE
2018-01-25, BUY EXECUTED, 151199000.00
2018-01-25, SELL CREATE
2018-01-25, SELL EXECUTED, 153507000.00
2018-01-25, BUY CREATE
2018-01-25, BUY EXECUTED, 152947000.00
2018-01-25, SELL CREATE
2018-01-25, SELL EXECUTED, 154474000.00
2018-01-25, BUY CREATE
2018-01-25, BUY EXECUTED, 153565000.00
2018-01-25, SELL CREATE
2018-01-25, SELL EXECUTED, 153581000.00
2018-01-25, BUY CREATE
2018-01-25, BUY EXECUTED, 153001000.00
2018-01-25, SELL CREATE
2018-01-25, SELL EXECUTED, 153256000.00
2018-01-25, BUY CREATE
2018-01-25, BUY EXECUTED, 151463000.00
2018-01-25, SELL CREATE
2018-01-25, SELL EXECUTED, 152651000.00
2018-01-25, BUY CREATE
2018-01-26, BUY EXECUTED, 150976000.00
2018-01-26, SELL CREATE
2018-01-26, SELL EXECUTED, 154001000.00
2018-01-26, BUY CREATE
2018-01-26, BUY EXECUTED, 153964000.00
2018-01-26, SELL CREATE
2018-01-26, SELL EXECUTED, 155000000.00
2018-01-26, BUY CREATE
2018-01-26, BUY EXECUTED, 154373000.00
2018-01-26, SELL

...

2018-12-09, BUY EXECUTED, 50398000.00
2018-12-09, SELL CREATE
2018-12-09, SELL EXECUTED, 50486000.00
2018-12-09, BUY CREATE
2018-12-09, BUY EXECUTED, 50001000.00
2018-12-09, SELL CREATE
2018-12-09, SELL EXECUTED, 50023000.00
2018-12-09, BUY CREATE
2018-12-09, BUY EXECUTED, 49900000.00
2018-12-09, SELL CREATE
2018-12-09, SELL EXECUTED, 50050000.00
2018-12-09, BUY CREATE
2018-12-09, BUY EXECUTED, 49999000.00
2018-12-09, SELL CREATE
2018-12-09, SELL EXECUTED, 50509000.00
2018-12-09, BUY CREATE
2018-12-09, BUY EXECUTED, 50424000.00
2018-12-09, SELL CREATE
2018-12-09, SELL EXECUTED, 50425000.00
2018-12-09, BUY CREATE
2018-12-09, BUY EXECUTED, 50315000.00
2018-12-09, SELL CREATE
2018-12-09, SELL EXECUTED, 51657000.00
2018-12-09, BUY CREATE
2018-12-09, BUY EXECUTED, 51466000.00
2018-12-09, SELL CREATE
2018-12-09, SELL EXECUTED, 51528000.00
2018-12-09, BUY CREATE
2018-12-09, BUY EXECUTED, 51395000.00
2018-12-09, SELL CREATE
2018-12-09, SELL EXECUTED, 52679000.00
2018-12-09, BUY CREATE
2018-12-

2018-12-13, BUY EXECUTED, 49703000.00
2018-12-13, SELL CREATE
2018-12-13, SELL EXECUTED, 49823000.00
2018-12-13, BUY CREATE
2018-12-13, BUY EXECUTED, 49566000.00
2018-12-13, SELL CREATE
2018-12-13, SELL EXECUTED, 49685000.00
2018-12-13, BUY CREATE
2018-12-13, BUY EXECUTED, 49337000.00
2018-12-13, SELL CREATE
2018-12-13, SELL EXECUTED, 49518000.00
2018-12-13, BUY CREATE
2018-12-13, BUY EXECUTED, 49500000.00
2018-12-13, SELL CREATE
2018-12-13, SELL EXECUTED, 49617000.00
2018-12-13, BUY CREATE
2018-12-13, BUY EXECUTED, 49614000.00
2018-12-13, SELL CREATE
2018-12-13, SELL EXECUTED, 49674000.00
2018-12-13, BUY CREATE
2018-12-13, BUY EXECUTED, 49637000.00
2018-12-13, SELL CREATE
2018-12-13, SELL EXECUTED, 49700000.00
2018-12-13, BUY CREATE
2018-12-13, BUY EXECUTED, 49630000.00
2018-12-13, SELL CREATE
2018-12-13, SELL EXECUTED, 49950000.00
2018-12-13, BUY CREATE
2018-12-13, BUY EXECUTED, 49875000.00
2018-12-13, SELL CREATE
2018-12-13, SELL EXECUTED, 49877000.00
2018-12-13, BUY CREATE
2018-12-

2018-12-19, BUY EXECUTED, 54101000.00
2018-12-19, SELL CREATE
2018-12-19, SELL EXECUTED, 54390000.00
2018-12-19, BUY CREATE
2018-12-19, BUY EXECUTED, 53861000.00
2018-12-19, SELL CREATE
2018-12-19, SELL EXECUTED, 53884000.00
2018-12-19, BUY CREATE
2018-12-19, BUY EXECUTED, 53595000.00
2018-12-19, SELL CREATE
2018-12-19, SELL EXECUTED, 53792000.00
2018-12-19, BUY CREATE
2018-12-19, BUY EXECUTED, 53732000.00
2018-12-19, SELL CREATE
2018-12-19, SELL EXECUTED, 53943000.00
2018-12-19, BUY CREATE
2018-12-19, BUY EXECUTED, 53901000.00
2018-12-19, SELL CREATE
2018-12-19, SELL EXECUTED, 54499000.00
2018-12-19, BUY CREATE
2018-12-19, BUY EXECUTED, 54450000.00
2018-12-19, SELL CREATE
2018-12-19, SELL EXECUTED, 54485000.00
2018-12-19, BUY CREATE
2018-12-19, BUY EXECUTED, 54295000.00
2018-12-19, SELL CREATE
2018-12-19, SELL EXECUTED, 55195000.00
2018-12-19, BUY CREATE
2018-12-19, BUY EXECUTED, 55077000.00
2018-12-19, SELL CREATE
2018-12-19, SELL EXECUTED, 55699000.00
2018-12-19, BUY CREATE
2018-12-

2018-12-23, SELL EXECUTED, 58998000.00
2018-12-23, BUY CREATE
2018-12-23, BUY EXECUTED, 58450000.00
2018-12-23, SELL CREATE
2018-12-23, SELL EXECUTED, 58483000.00
2018-12-23, BUY CREATE
2018-12-23, BUY EXECUTED, 57791000.00
2018-12-23, SELL CREATE
2018-12-23, SELL EXECUTED, 58041000.00
2018-12-23, BUY CREATE
2018-12-23, BUY EXECUTED, 57934000.00
2018-12-23, SELL CREATE
2018-12-23, SELL EXECUTED, 58281000.00
2018-12-23, BUY CREATE
2018-12-24, BUY EXECUTED, 58004000.00
2018-12-24, SELL CREATE
2018-12-24, SELL EXECUTED, 58086000.00
2018-12-24, BUY CREATE
2018-12-24, BUY EXECUTED, 57864000.00
2018-12-24, SELL CREATE
2018-12-24, SELL EXECUTED, 58095000.00
2018-12-24, BUY CREATE
2018-12-24, BUY EXECUTED, 57970000.00
2018-12-24, SELL CREATE
2018-12-24, SELL EXECUTED, 58356000.00
2018-12-24, BUY CREATE
2018-12-24, BUY EXECUTED, 57940000.00
2018-12-24, SELL CREATE
2018-12-24, SELL EXECUTED, 57951000.00
2018-12-24, BUY CREATE
2018-12-24, BUY EXECUTED, 57853000.00
2018-12-24, SELL CREATE
2018-12-

2018-12-28, BUY EXECUTED, 53574000.00
2018-12-28, SELL CREATE
2018-12-28, SELL EXECUTED, 53690000.00
2018-12-28, BUY CREATE
2018-12-28, BUY EXECUTED, 53619000.00
2018-12-28, SELL CREATE
2018-12-28, SELL EXECUTED, 53841000.00
2018-12-28, BUY CREATE
2018-12-28, BUY EXECUTED, 53799000.00
2018-12-28, SELL CREATE
2018-12-28, SELL EXECUTED, 53880000.00
2018-12-28, BUY CREATE
2018-12-28, BUY EXECUTED, 53721000.00
2018-12-28, SELL CREATE
2018-12-28, SELL EXECUTED, 54198000.00
2018-12-28, BUY CREATE
2018-12-28, BUY EXECUTED, 54135000.00
2018-12-28, SELL CREATE
2018-12-28, SELL EXECUTED, 54400000.00
2018-12-28, BUY CREATE
2018-12-28, BUY EXECUTED, 54248000.00
2018-12-28, SELL CREATE
2018-12-28, SELL EXECUTED, 54494000.00
2018-12-28, BUY CREATE
2018-12-28, BUY EXECUTED, 54220000.00
2018-12-28, SELL CREATE
2018-12-28, SELL EXECUTED, 54255000.00
2018-12-28, BUY CREATE
2018-12-28, BUY EXECUTED, 54221000.00
2018-12-28, SELL CREATE
2018-12-28, SELL EXECUTED, 54326000.00
2018-12-28, BUY CREATE
2018-12-